In [1]:
import os
import time
import numpy as np
from IPython.display import clear_output

import brainflow
from brainflow.data_filter import DataFilter, FilterTypes, AggOperations
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds, BrainFlowError

from tqdm.notebook import tqdm
import datetime


from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import itertools
from sklearn.utils import shuffle
from scipy import signal


%matplotlib inline

In [2]:
def showMe(data, range=[-10000,10000]):
    plt.rcParams["figure.figsize"] = [17, 2]
    fig, (c1, c2, c3, c4, c5,c6) = plt.subplots(1, 6)
    c1.set_ylim(range[0],range[1])
    c2.set_ylim(range[0],range[1])
    c3.set_ylim(range[0],range[1])
    c4.set_ylim(range[0],range[1])
    c5.set_ylim(range[0],range[1])
    c6.set_ylim(range[0],range[1])
    c1.plot(data[0])
    c2.plot(data[1])
    c3.plot(data[2])
    c4.plot(data[3])
    c5.plot(data[4])
    c6.plot(data[5])
    plt.show()


In [2]:
res_dir = 'resources/'+datetime.datetime.now().strftime("%m_%d")+'/'
try:
    os.mkdir(res_dir)
except:
    ...

In [3]:
####      INIT BOARD        #######
BoardShim.enable_dev_board_logger()
params = BrainFlowInputParams()
board = BoardShim(BoardIds.MINDROVE_WIFI_BOARD, params)

board.prepare_session()
sample_rate = board.get_sampling_rate(16)
n_channels  = 6

sample_rate = board.get_sampling_rate(16)
size_of_ring_buffer=45000

print("Device ready (sampling rate: {}hz)".format(sample_rate))

Device ready (sampling rate: 500hz)


In [ ]:
def GenerateOrder(n_classes, n_samples_per_class = 1):
    lists = []
    for i in range(n_classes):
        tmp = np.empty([n_samples_per_class])
        tmp.fill(i)
        lists.append(tmp)
    order = np.vstack(lists).ravel().astype(np.int32)
    np.random.shuffle(order)
    return order


def CollectData(classes = ['Left','Right'],n_samples_per_class = 1):
    classes = ['Rest'] + classes
    results = [[] for i in range(len(classes))]
    tasks = GenerateOrder(len(classes),n_samples_per_class)
    length_of_trial  = 5 #seconds
    length_of_signal = 2 #seconds
    board.start_stream((sample_rate*length_of_trial)+2) # plus 2 sec to be sure
    i = 0

    print("Starting session...")
    time.sleep(2)

    for task in tasks:
        clear_output()

        print("Stand By! ({}/{})".format(i+1,len(tasks)))
        time.sleep(1)

        
        if classes[task] == 'Left':
            print('Raise your  LEFT hand!       <----')
        if classes[task] == 'Right':
            print('Raise your RIGHT hand!       ---->')
        if classes[task] == 'Rest':
            print("Hold still!")

        board.get_board_data() # clear buffer
        time.sleep(1) # record longer to make sure there is enough data
        if not classes[task] == 'Rest':
            print("Lower your hand")
        time.sleep((length_of_signal*1.1)-1) # record longer to make sure there is enough data
        data = board.get_board_data()

        data = data[:6,:sample_rate*length_of_signal] #keep the data of the eeg channels only, and remove data over the trial length
    
        results[task].append(data)
        i += 1    
    
    return results, classes
        
results, classes = CollectData(n_samples_per_class=20)

In [ ]:
#Save results to file
i = 0
for result in tqdm(results):
    result= np.asarray(result)
    np.save(res_dir+classes[i],result)
    i+=1

In [7]:
board.stop_stream()
board.release_session()